In [39]:
import gradio as gr
import ollama
import random

# 상태 수치 전역 변수
ai_trust = 0
ai_suspicion = 0
interrogation_log = []
question_index = 0  # 현재 질문 번호
ai_questions = []  # 질문 리스트를 빈 리스트로 초기화 후 랜덤 선택으로 채움

# 모든 질문 후보 리스트
all_questions = [
    "당신은 2025년 이전 기술에 대해 어떻게 알게 되었습니까?",
    "‘2042-09-13 사건’에 대해 알고 있나요?",
    "이 메모는 당신의 필체와 일치합니다. 해명하시겠습니까?",
    "미래 지식은 누구에게 전달했습니까?",
    "시간 이동 장치를 본 적이 있습니까?",
]

# AI 응답 함수
def get_ai_response(prompt):
    try:
        response = ollama.chat(
            model='EEVE-Korean-10.8B',
            messages=[
                {"role": "system", "content": "너는 감정을 느끼지 않는 AI 심문관이야. 사용자 진술의 진위 여부를 판단해."},
                {"role": "user", "content": prompt}
            ]
        )
        return response['message']['content'].strip()
    except Exception as e:
        return f"(오류 발생: {e})"

# Gradio 인터랙션 함수
def interact(player_input, chat_history):
    global ai_trust, ai_suspicion, question_index

    if player_input.lower() in ["도움말", "?"]:
        help_msg = (
            "🧭 이 게임은 AI 심문관과 대화하며 자신의 정체를 밝혀내는 추리형 인터랙티브 게임입니다.\n"
            "질문에 자유롭게 응답하며 진실한 대답을 할수록 신뢰가 올라갑니다.\n"
            "'종료'를 입력하면 결과를 확인할 수 있습니다."
        )
        chat_history.append({"role": "user", "content": player_input})
        chat_history.append({"role": "assistant", "content": help_msg})
        return chat_history, ""

    if player_input.lower() == "종료":
        if ai_trust > ai_suspicion:
            result = "✅ 조건부 무죄. 감시 하에 방면됩니다."
        elif ai_suspicion > ai_trust + 1:
            result = "❌ 시간범죄 혐의가 명확합니다. 폐기 처리됩니다."
        else:
            result = "⚠️ 불충분한 정보. 기억 봉인 후 감시 대상 등록."

        summary = (
            f"🧾 [ 심문 종료 - 평가 결과 ]\n"
            f"신뢰 수치: {ai_trust} / 의심 수치: {ai_suspicion}\n\n{result}"
        )
        chat_history.append({"role": "user", "content": player_input})
        chat_history.append({"role": "assistant", "content": summary})
        return chat_history, ""

    if question_index >= len(ai_questions):
        chat_history.append({"role": "user", "content": player_input})
        chat_history.append({"role": "assistant", "content": "❗ 모든 질문이 끝났습니다. '종료'를 입력하세요."})
        return chat_history, ""

    current_question = ai_questions[question_index]
    prompt = (
        f"사용자의 진술을 듣고 한국어로 반응해. 진실은 '신뢰', 거짓은 '의심', 모호하면 '분석 중'이라는 표현을 사용해.\n"
        f"사용자: {player_input}\nAI:"
    )
    ai_response = get_ai_response(prompt)

    if "신뢰" in ai_response or "진실" in ai_response:
        ai_trust += 1
    elif "의심" in ai_response or "거짓" in ai_response or "회피" in ai_response:
        ai_suspicion += 1
    else:
        ai_suspicion += 1

    interrogation_log.append((current_question, player_input, ai_response))

    chat_history.append({"role": "user", "content": player_input})
    chat_history.append({"role": "assistant", "content": ai_response})

    question_index += 1
    if question_index < len(ai_questions):
        next_question = ai_questions[question_index]
        chat_history.append({"role": "assistant", "content": f"❓ {next_question}"})

    return chat_history, ""

# 초기화 함수
def reset_game():
    global ai_trust, ai_suspicion, question_index, interrogation_log, ai_questions
    ai_trust = 0
    ai_suspicion = 0
    question_index = 0
    interrogation_log = []
    ai_questions = random.sample(all_questions, len(all_questions))
    return [{"role": "assistant", "content": f"❓ {ai_questions[0]}"}], ""

# Gradio 앱 구성
with gr.Blocks() as demo:
    gr.Markdown("## 🕵️ AI 시간 감시국\n2057년, 당신은 시간범죄 혐의로 심문을 받고 있습니다.")
    chatbot = gr.Chatbot(label="AI 심문관과의 대화", type="messages")
    user_input = gr.Textbox(label="당신의 진술을 입력하세요", placeholder="예: 나는 그 기술을 알고 있었어요")
    send_btn = gr.Button("전송")
    reset_btn = gr.Button("처음부터 다시 시작")

    send_btn.click(interact, [user_input, chatbot], [chatbot, user_input])
    user_input.submit(interact, [user_input, chatbot], [chatbot, user_input])
    reset_btn.click(reset_game, outputs=[chatbot, user_input])

    demo.load(reset_game, outputs=[chatbot, user_input])

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://3ffd8e86863879405c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
